# Reduction of the memory footprint?

We examine the memory footprint of TF corpora and test a packed representation of the same data.

We measure the memory usage and the performance of accesing feature values.

We also take into account the pre-computed features `levUp` and `levDown`.

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import collections
from timeit import timeit
from tf.app import use
from tf.fabric import Fabric
from tf.applib.helpers import dm
from pack import deepSize, pack, Pack

In [10]:
def testValues(origData, packData, maxNode):
  isDict = packData.container == 'dict'
  error = 0
  for n in range(1, maxNode + 1):
    oe = origData.get(n, None) if isDict else origData[n] if n < len(origData) else None
    pe = packData.get(n)
    if oe != pe:
      error +=1
  return error
  
def testPerformance(origData, packData):
  isDict = packData.container == 'dict'
  times = 1000
  testMember = list(k for (k, v) in origData.items() if v is not None)[0] if isDict else 0
  origTime = timeit("origData[testMember]", globals=locals(), number=times)
  packedTime = timeit("packData.get(testMember)", globals=locals(), number=times)
  return packedTime / origTime if origTime else 0

CMAP = dict(
    oslots='tup',
    otype='tup',
    __levUp__='tup',
    __levDown__='tup',
)
EMAP = dict(
    oslots='tup',
    otype='str',
    __levUp__='tup',
    __levDown__='tup',
)
  
def testFeature(TF, ft):
  api = TF.api
  F = api.F
  maxNode = F.otype.maxNode
  
  info = TF.features[ft]
  origData = info.data
  if not origData:
    return
  if ft == 'otype':
    origData = origData[0:-2]
  elif ft == 'oslots':
    origData = origData[0:-1]
  containerType = CMAP.get(ft, 'dict')
  elementType = EMAP.get(ft, getattr(info, 'metaData', {}).get('valueType', None))
  if ft != 'oslots' and info.isEdge:
    origData = {n: tuple(ns) for (n, ns) in origData.items() if ns is not None}
    elementType = 'tup'
  
  packDataRaw = pack(origData, containerType, elementType)
  packData = Pack(packDataRaw)
  
  isDict = packData.container == 'dict'
  
  fType = f'{containerType}:{elementType}'
  origLen = len(origData)
  packLen = packData.length
  gapLen = packData.alength - packLen if isDict else 0
  origSize = deepSize(origData)
  packSize = deepSize(packDataRaw)
  sizeRatio = int(round(100 * packSize / origSize)) if origSize else 0
  origAvSize = int(round(origSize / origLen)) if origLen else 0
  packAvSize = int(round(packSize / packLen)) if packLen else 0
  perfRatio = testPerformance(origData, packData)
  errors = testValues(origData, packData, maxNode)
  
  return (
      ft,
      fType,
      origLen,
      packLen,
      origSize,
      origAvSize,
      packSize,
      packAvSize,
      sizeRatio,
      perfRatio,
      errors,
  )

In [11]:
FIELDS = '''
    name
    type
    lengthOrig
    lengthPack
    sizeOrig
    elemSizeOrig
    sizePack
    elemSizePack
    sizeRatio
    perfRatio
    errors
'''.strip().split()

SUMMABLE = set('''
  lengthOrig
  lengthPack
  sizeOrig
  sizePack
  perfRatio
  errors
'''.strip().split())

AVERAGEABLE = dict(
  elemSizeOrig = ('sizeOrig', 'lengthOrig'),
  elemSizePack = ('sizePack', 'lengthPack'),
  sizeRatio = ('sizePack', 'sizeOrig'),
)

ROWFMT = (
  '`{name}` | '
  '{type} | '
  '{lengthOrig:,} | '
  '{lengthPack:,} | '
  '{errors:,} '
  '{elemSizeOrig:,} | '
  '{elemSizePack:,} | '
  '{sizeOrig:,} | '
  '{sizePack:,} | '
  '{sizeRatio}% | '
  '{perfRatio:4.1f}x | '
  '\n'
)
HEADFMT = (
  '{name} | '
  '{type} | '
  '{lengthOrig} | '
  '{lengthPack} | '
  '{errors}'
  '{elemSizeOrig} | '
  '{elemSizePack} | '
  '{sizeOrig} | '
  '{sizePack} | '
  '{sizeRatio} | '
  '{perfRatio} | '
  '\n'
)


TABLEHDR =  (
  HEADFMT.format(**{f: f for f in FIELDS}) +
  HEADFMT.format(**{f: '---' for f in FIELDS})
)

In [12]:
def testFeatures(corpus, fabric=False):
  if fabric:
    TF = Fabric(locations=corpus, silent=True)
    api = TF.load('', silent=True)
    allFeatures = TF.explore(silent=True, show=True)
    loadableFeatures = allFeatures['nodes'] + allFeatures['edges']
    TF.load(loadableFeatures, add=True, silent=True)
  else:
    A = use(f'{corpus}:clone', checkout='clone', silent=True)
    api = A.api
    TF = api.TF
  F = api.F
  features = TF.features
  totals = collections.Counter()
  
  material = ''
  
  n = 0
  for ft in sorted(features):
    if ft.startswith('__') and not (ft in CMAP or ft in EMAP):
      continue
    if ft == 'otext':
      continue
    sys.stdout.write(f'\r{ft:<30}')
    results = testFeature(TF, ft)
    if not results:
      continue
    n += 1
    hresults = dict(zip(FIELDS, results))
    material += ROWFMT.format(**hresults)
    for s in SUMMABLE:
      totals[s] += hresults[s]
  print(f'\r{"":>40}')
  for (a, (nm, dnm)) in AVERAGEABLE.items():
    theNm = totals[nm]
    theDnm = totals[dnm]
    totals[a] = theNm / theDnm if theDnm else 0
  totals['perfRatio'] = totals['perfRatio'] / n if n else 0
  for s in ('elemSizeOrig', 'elemSizePack'):
    totals[s] = int(round(totals[s]))
  for s in ('sizeRatio',):
    totals[s] = int(round(100 * totals[s]))
  material += ROWFMT.format(name='TOTAL', type='ALL', **totals)
  header = f'''
# {A.repo} {n} features

{TABLEHDR}'''
  
  dm(header + material)

In [18]:
def footPrint(corpus, name=None, fabric=False):
  if fabric:
    if name is None:
      name = os.path.basename(name)
    TF = Fabric(locations=corpus, silent=True)
    api = TF.load('', silent=True)
    allFeatures = TF.explore(silent=True, show=True)
    loadableFeatures = allFeatures['nodes'] + allFeatures['edges']
    TF.load(loadableFeatures, add=True, silent=True)
  else:
    A = use(f'{corpus}:clone', checkout='clone', silent=True)
    api = A.api
    TF = api.TF
    if name is None:
      name = A.repo
  F = api.F
  features = TF.features
  totals = collections.Counter()
  
  material = ''
  
  n = 0
  for ft in sorted(features):
    sys.stdout.write(f'\r{ft:<30}')
    data = features[ft].data
    if data is None:
      continue
    n += 1
    nData = len(data)
    sData = deepSize(data)
    material += f'{ft} | {nData:,} | {sData:,}\n'
    totals['nData'] += nData
    totals['sData'] += sData
  print(f'\r{"":>40}')
  material += f'TOTAL | {totals["nData"]:,} | {totals["sData"]:,}'
  header = f'''
# {name} {n} features

feature | members | size in bytes
--- | --- | ---
'''
  
  dm(header + material)

In the table below you can sea how much bytes in memory the features of our corpora need.

Here is the legend:

Features:

* levDown: is what powers `L.d()`
* levUp: is what powers `L.u()`

Columns

* name: of the feature
* type: whether the feature is stored is a dict or as a tuple, and whether its elements are integers or strings
* length[Orig/Pack]: the number of values, both in the orginal and the packed version (should be equal)
* errors: all values in the original and packed representations have been compared. This is the number of mismatches, if any
* elemSize[Orig/Pack]: how much bytes a value needs to store it on average for this feature
* size[Orig/Pack]: the total amount of bytes in memory that the whole feature needs
* sizeRatio: the success of packing: how much percent the packed version needs as compared to the original version
* perfRatio: the performance: how much more time accessing values cost using the packed representation as compared to the original one

# Conclusion

We can gain a more than 10 fold reduction of the memory footprint for the big corpora, but at the cost of a 20 fold
performance hit.

That is not worth it for now, because queries will become very sluggish.

However, we can make a lesser optimization, without any impact on the performance of accessing feature values.
This has to do with sharing Python objects.

After feature construction, we will make sure that all values of a feature share as many objects as possible.

This will give a reduction of 33% for the BHSA, which is pleasant for those who work on a 4GB machine.

See "Testing mild memory reduction" below

# Testing aggressive memory reduction

In [10]:
testFeatures('banks')


# banks 11 features

name | type | lengthOrig | lengthPack | errorselemSizeOrig | elemSizePack | sizeOrig | sizePack | sizeRatio | perfRatio | 
--- | --- | --- | --- | ------ | --- | --- | --- | --- | --- | 
`__levDown__` | tup:tup | 18 | 18 | 0 76 | 84 | 1,364 | 1,507 | 110% | 18.0x | 
`__levUp__` | tup:tup | 117 | 117 | 0 90 | 17 | 10,576 | 2,043 | 19% | 15.5x | 
`author` | dict:str | 1 | 1 | 0 330 | 1,408 | 330 | 1,408 | 427% | 19.7x | 
`gap` | dict:int | 7 | 7 | 0 85 | 247 | 592 | 1,728 | 292% |  8.8x | 
`letters` | dict:str | 99 | 99 | 0 130 | 25 | 12,826 | 2,474 | 19% | 20.9x | 
`number` | dict:int | 17 | 17 | 0 79 | 92 | 1,348 | 1,568 | 116% |  9.0x | 
`oslots` | tup:tup | 18 | 18 | 0 386 | 158 | 6,940 | 2,843 | 41% | 32.5x | 
`otype` | tup:str | 18 | 18 | 0 23 | 76 | 411 | 1,368 | 333% | 19.1x | 
`punc` | dict:str | 17 | 17 | 0 78 | 108 | 1,324 | 1,836 | 139% | 20.7x | 
`terminator` | dict:str | 12 | 12 | 0 99 | 124 | 1,184 | 1,492 | 126% | 19.6x | 
`title` | dict:str | 1 | 1 | 0 333 | 1,417 | 333 | 1,417 | 426% | 21.0x | 
`TOTAL` | ALL | 325 | 325 | 0 115 | 61 | 37,228 | 19,684 | 53% | 18.6x | 


In [11]:
testFeatures('dss')


# dss 59 features

name | type | lengthOrig | lengthPack | errorselemSizeOrig | elemSizePack | sizeOrig | sizePack | sizeRatio | perfRatio | 
--- | --- | --- | --- | ------ | --- | --- | --- | --- | --- | 
`__levDown__` | tup:tup | 677,618 | 677,618 | 0 170 | 22 | 114,895,720 | 15,213,171 | 13% | 13.8x | 
`__levUp__` | tup:tup | 2,107,856 | 2,107,856 | 0 235 | 17 | 494,912,728 | 35,435,447 | 7% | 16.7x | 
`after` | dict:str | 798,546 | 798,546 | 0 81 | 10 | 64,302,482 | 7,973,524 | 12% | 21.2x | 
`alt` | dict:int | 577 | 577 | 0 60 | 4,643 | 34,712 | 2,678,724 | 7717% |  8.7x | 
`biblical` | dict:int | 267,671 | 267,671 | 0 67 | 6 | 17,980,732 | 1,649,896 | 9% |  8.7x | 
`book` | dict:str | 218,186 | 218,186 | 0 128 | 4 | 27,923,866 | 901,403 | 3% | 20.4x | 
`chapter` | dict:str | 218,186 | 218,186 | 0 111 | 4 | 24,153,921 | 902,958 | 4% | 22.2x | 
`cl` | dict:str | 312,839 | 312,839 | 0 114 | 7 | 35,669,337 | 2,101,729 | 6% | 11.8x | 
`cl2` | dict:str | 1,334 | 1,334 | 0 57 | 1,396 | 76,598 | 1,862,388 | 2431% | 19.9x | 
`cor` | dict:int | 5,698 | 5,698 | 0 80 | 973 | 454,636 | 5,543,104 | 1219% |  8.8x | 
`fragment` | dict:str | 64,077 | 64,077 | 0 117 | 4 | 7,498,336 | 281,193 | 4% | 19.3x | 
`full` | dict:str | 500,992 | 500,992 | 0 151 | 7 | 75,471,577 | 3,550,066 | 5% | 21.2x | 
`fulle` | dict:str | 500,992 | 500,992 | 0 113 | 6 | 56,845,294 | 3,132,318 | 6% | 19.6x | 
`fullo` | dict:str | 500,992 | 500,992 | 0 112 | 6 | 56,022,086 | 3,021,020 | 5% | 19.1x | 
`glex` | dict:str | 481,281 | 481,281 | 0 153 | 5 | 73,637,279 | 2,225,669 | 3% | 22.2x | 
`glexe` | dict:str | 481,281 | 481,281 | 0 122 | 5 | 58,501,283 | 2,178,880 | 4% | 20.6x | 
`glexo` | dict:str | 481,281 | 481,281 | 0 125 | 5 | 60,149,680 | 2,228,607 | 4% | 22.8x | 
`glyph` | dict:str | 1,838,363 | 1,838,363 | 0 146 | 5 | 268,732,701 | 8,417,824 | 3% | 21.9x | 
`glyphe` | dict:str | 1,838,363 | 1,838,363 | 0 86 | 4 | 157,531,402 | 8,250,695 | 5% | 22.1x | 
`glypho` | dict:str | 1,838,363 | 1,838,363 | 0 84 | 4 | 154,846,302 | 8,241,506 | 5% | 22.3x | 
`gn` | dict:str | 194,777 | 194,777 | 0 82 | 11 | 15,997,164 | 2,101,600 | 13% | 19.3x | 
`gn2` | dict:str | 44,408 | 44,408 | 0 87 | 47 | 3,866,350 | 2,101,764 | 54% | 19.5x | 
`gn3` | dict:str | 14 | 14 | 0 85 | 12,605 | 1,190 | 176,472 | 14830% | 19.9x | 
`halfverse` | dict:str | 932 | 932 | 0 68 | 34 | 63,164 | 31,728 | 50% | 19.6x | 
`intl` | dict:int | 1,632 | 1,632 | 0 73 | 1,853 | 119,576 | 3,023,824 | 2529% |  8.9x | 
`lang` | dict:str | 185,139 | 185,139 | 0 85 | 22 | 15,669,856 | 4,093,932 | 26% |  9.6x | 
`lex` | dict:str | 481,288 | 481,288 | 0 150 | 5 | 72,316,950 | 2,267,061 | 3% | 22.0x | 
`lexe` | dict:str | 481,288 | 481,288 | 0 125 | 5 | 60,122,064 | 2,211,338 | 4% | 20.2x | 
`lexo` | dict:str | 481,288 | 481,288 | 0 128 | 5 | 61,377,459 | 2,211,338 | 4% | 20.0x | 
`line` | dict:str | 52,895 | 52,895 | 0 90 | 4 | 4,752,297 | 224,550 | 5% | 18.5x | 
`md` | dict:str | 1,984 | 1,984 | 0 118 | 939 | 234,534 | 1,862,397 | 794% | 19.9x | 
`merr` | dict:str | 2 | 2 | 0 201 | 772 | 402 | 1,544 | 384% | 21.0x | 
`morpho` | dict:str | 470,842 | 470,842 | 0 117 | 5 | 54,871,512 | 2,124,595 | 4% | 20.4x | 
`nu` | dict:str | 194,670 | 194,670 | 0 82 | 11 | 15,997,366 | 2,101,600 | 13% | 19.4x | 
`nu2` | dict:str | 44,406 | 44,406 | 0 87 | 47 | 3,865,684 | 2,101,764 | 54% | 20.1x | 
`nu3` | dict:str | 14 | 14 | 0 78 | 12,604 | 1,090 | 176,456 | 16189% | 19.9x | 
`occ` | dict:tup | 10,449 | 10,449 | 0 1,726 | 198 | 18,039,336 | 2,063,768 | 11% | 16530.7x | 
`oslots` | tup:tup | 677,618 | 677,618 | 0 466 | 53 | 316,031,972 | 35,620,951 | 11% | 15.3x | 
`otype` | tup:str | 677,618 | 677,618 | 0 8 | 4 | 5,421,318 | 2,845,245 | 52% | 19.3x | 
`ps` | dict:str | 49,100 | 49,100 | 0 82 | 43 | 4,024,718 | 2,102,092 | 52% | 19.6x | 
`ps2` | dict:str | 44,409 | 44,409 | 0 87 | 47 | 3,865,146 | 2,101,752 | 54% | 19.6x | 
`ps3` | dict:str | 14 | 14 | 0 85 | 12,605 | 1,190 | 176,472 | 14830% | 19.4x | 
`punc` | dict:str | 29,927 | 29,927 | 0 148 | 70 | 4,423,260 | 2,101,576 | 48% | 21.9x | 
`punce` | dict:str | 29,927 | 29,927 | 0 123 | 70 | 3,675,085 | 2,101,576 | 57% | 20.5x | 
`punco` | dict:str | 29,927 | 29,927 | 0 72 | 70 | 2,148,896 | 2,101,572 | 98% | 19.6x | 
`rec` | dict:int | 661,786 | 661,786 | 0 60 | 8 | 39,501,652 | 5,542,940 | 14% |  8.3x | 
`rem` | dict:int | 2,264 | 2,264 | 0 61 | 2,304 | 137,272 | 5,217,168 | 3801% |  8.7x | 
`script` | dict:str | 4,110 | 4,110 | 0 124 | 1,195 | 509,263 | 4,910,389 | 964% | 19.9x | 
`scroll` | dict:str | 65,078 | 65,078 | 0 122 | 4 | 7,945,465 | 277,710 | 3% | 20.4x | 
`sim` | dict:tup | 10,732 | 10,732 | 0 277 | 44 | 2,977,600 | 473,524 | 16% | 17.6x | 
`sp` | dict:str | 470,841 | 470,841 | 0 126 | 4 | 59,350,493 | 2,101,660 | 4% | 18.8x | 
`srcLn` | dict:int | 500,992 | 500,992 | 0 98 | 7 | 49,021,176 | 3,475,284 | 7% |  8.1x | 
`st` | dict:str | 107,481 | 107,481 | 0 77 | 20 | 8,279,250 | 2,101,600 | 25% | 19.2x | 
`type` | dict:str | 2,032,329 | 2,032,329 | 0 123 | 4 | 249,123,881 | 8,471,969 | 3% | 21.5x | 
`unc` | dict:int | 92,266 | 92,266 | 0 85 | 64 | 7,826,536 | 5,889,484 | 75% |  8.8x | 
`vac` | dict:int | 3,524 | 3,524 | 0 70 | 1,671 | 246,232 | 5,889,464 | 2392% |  8.9x | 
`verse` | dict:str | 932 | 932 | 0 68 | 34 | 63,164 | 31,728 | 50% | 19.6x | 
`vs` | dict:str | 66,332 | 66,332 | 0 120 | 32 | 7,976,057 | 2,102,485 | 26% | 20.5x | 
`vt` | dict:str | 66,317 | 66,317 | 0 121 | 32 | 7,994,538 | 2,102,152 | 26% | 20.3x | 
`TOTAL` | ALL | 21,404,048 | 21,404,048 | 0 134 | 11 | 2,857,480,830 | 242,404,646 | 8% | 297.8x | 


In [15]:
testFeatures('oldbabylonian')


# oldbabylonian 68 features

name | type | lengthOrig | lengthPack | errorselemSizeOrig | elemSizePack | sizeOrig | sizePack | sizeRatio | perfRatio | 
--- | --- | --- | --- | ------ | --- | --- | --- | --- | --- | 
`ARK` | dict:str | 853 | 853 | 0 136 | 28 | 116,297 | 24,059 | 21% | 26.0x | 
`__levDown__` | tup:tup | 131,448 | 131,448 | 0 142 | 19 | 18,650,928 | 2,468,915 | 13% | 15.5x | 
`__levUp__` | tup:tup | 334,667 | 334,667 | 0 191 | 13 | 63,973,768 | 4,333,435 | 7% | 16.9x | 
`after` | dict:str | 219,124 | 219,124 | 0 76 | 5 | 16,621,636 | 1,146,784 | 7% | 19.4x | 
`afterr` | dict:str | 175,844 | 175,844 | 0 91 | 5 | 15,928,040 | 847,076 | 5% | 20.0x | 
`afteru` | dict:str | 50,122 | 50,122 | 0 80 | 17 | 4,025,210 | 847,040 | 21% | 19.7x | 
`atf` | dict:str | 279,722 | 279,722 | 0 112 | 6 | 31,387,619 | 1,545,860 | 5% | 19.5x | 
`atfpost` | dict:str | 21,969 | 21,969 | 0 91 | 39 | 2,004,847 | 847,144 | 42% | 19.7x | 
`atfpre` | dict:str | 21,769 | 21,769 | 0 59 | 39 | 1,287,609 | 847,140 | 66% | 19.8x | 
`author` | dict:str | 1,284 | 1,284 | 0 125 | 6 | 161,018 | 7,492 | 5% | 20.7x | 
`col` | dict:int | 289 | 289 | 0 61 | 265 | 17,496 | 76,448 | 437% |  9.6x | 
`collated` | dict:int | 13 | 13 | 0 80 | 325 | 1,040 | 4,220 | 406% |  8.8x | 
`collection` | dict:str | 1,284 | 1,284 | 0 108 | 5 | 139,252 | 6,933 | 5% | 20.9x | 
`comment` | dict:str | 971 | 971 | 0 128 | 729 | 123,960 | 708,338 | 571% | 20.0x | 
`damage` | dict:int | 9,974 | 9,974 | 0 58 | 80 | 574,308 | 797,752 | 139% |  8.9x | 
`det` | dict:int | 6,796 | 6,796 | 0 71 | 125 | 485,324 | 847,032 | 175% |  8.9x | 
`docnote` | dict:str | 108 | 108 | 0 133 | 45 | 14,367 | 4,897 | 34% | 20.7x | 
`docnumber` | dict:str | 1,202 | 1,202 | 0 111 | 7 | 133,039 | 8,990 | 7% | 20.8x | 
`excavation` | dict:str | 2 | 2 | 0 206 | 1,518 | 411 | 3,037 | 739% | 20.8x | 
`excised` | dict:int | 99 | 99 | 0 76 | 5,280 | 7,504 | 522,696 | 6966% |  8.9x | 
`face` | dict:str | 2,834 | 2,834 | 0 136 | 5 | 385,328 | 13,365 | 3% | 19.6x | 
`flags` | dict:str | 10,498 | 10,498 | 0 57 | 76 | 596,650 | 797,781 | 134% | 19.0x | 
`fraction` | dict:str | 121 | 121 | 0 119 | 5,501 | 14,384 | 665,672 | 4628% | 14.3x | 
`genre` | dict:str | 1,285 | 1,285 | 0 112 | 5 | 143,651 | 6,865 | 5% | 21.0x | 
`grapheme` | dict:str | 3,011 | 3,011 | 0 129 | 265 | 387,385 | 799,139 | 206% | 20.6x | 
`graphemer` | dict:str | 3,011 | 3,011 | 0 131 | 265 | 394,608 | 799,174 | 203% | 20.7x | 
`graphemeu` | dict:str | 3,011 | 3,011 | 0 142 | 265 | 427,402 | 799,224 | 187% | 20.6x | 
`lang` | dict:str | 1,285 | 1,285 | 0 109 | 5 | 139,768 | 6,792 | 5% | 20.0x | 
`langalt` | dict:int | 19,063 | 19,063 | 0 59 | 44 | 1,123,720 | 847,032 | 75% |  9.0x | 
`ln` | dict:int | 26,406 | 26,406 | 0 78 | 4 | 2,051,276 | 115,528 | 6% |  8.7x | 
`lnc` | dict:str | 969 | 969 | 0 66 | 106 | 64,150 | 102,452 | 160% | 19.7x | 
`lnno` | dict:str | 27,375 | 27,375 | 0 94 | 4 | 2,564,299 | 115,913 | 5% | 19.4x | 
`material` | dict:str | 1,285 | 1,285 | 0 110 | 5 | 141,077 | 6,844 | 5% | 26.3x | 
`missing` | dict:int | 14,391 | 14,391 | 0 69 | 59 | 992,876 | 847,192 | 85% |  8.6x | 
`museumcode` | dict:str | 1,285 | 1,285 | 0 115 | 19 | 147,528 | 24,269 | 16% | 21.3x | 
`museumname` | dict:str | 1,285 | 1,285 | 0 142 | 6 | 182,295 | 7,561 | 4% | 21.8x | 
`object` | dict:str | 2,834 | 2,834 | 0 135 | 5 | 382,846 | 13,110 | 3% | 19.7x | 
`operator` | dict:str | 122 | 122 | 0 67 | 4,033 | 8,220 | 492,040 | 5986% | 19.7x | 
`operatorr` | dict:str | 122 | 122 | 0 70 | 4,033 | 8,550 | 492,044 | 5755% | 19.0x | 
`operatoru` | dict:str | 122 | 122 | 0 70 | 4,033 | 8,550 | 492,044 | 5755% | 19.3x | 
`oslots` | tup:tup | 131,448 | 131,448 | 0 290 | 34 | 38,069,524 | 4,455,319 | 12% | 15.9x | 
`otype` | tup:str | 131,448 | 131,448 | 0 8 | 4 | 1,051,904 | 554,293 | 53% | 19.4x | 
`period` | dict:str | 1,285 | 1,285 | 0 139 | 5 | 178,774 | 6,861 | 4% | 20.8x | 
`pnumber` | dict:str | 1,285 | 1,285 | 0 113 | 17 | 144,908 | 21,633 | 15% | 20.9x | 
`primecol` | dict:int | 4 | 4 | 0 95 | 363 | 380 | 1,452 | 382% |  9.1x | 
`primeln` | dict:int | 1,825 | 1,825 | 0 68 | 50 | 124,952 | 91,216 | 73% |  8.9x | 
`pubdate` | dict:str | 1,285 | 1,285 | 0 110 | 5 | 141,488 | 7,051 | 5% | 21.1x | 
`question` | dict:int | 560 | 560 | 0 61 | 1,189 | 34,236 | 665,628 | 1944% |  8.9x | 
`reading` | dict:str | 199,359 | 199,359 | 0 124 | 4 | 24,719,785 | 852,859 | 3% | 19.4x | 
`readingr` | dict:str | 199,359 | 199,359 | 0 130 | 4 | 25,966,530 | 853,644 | 3% | 19.3x | 
`readingu` | dict:str | 199,359 | 199,359 | 0 157 | 4 | 31,335,193 | 850,344 | 3% | 20.3x | 
`remarkable` | dict:int | 99 | 99 | 0 76 | 5,280 | 7,504 | 522,696 | 6966% |  8.8x | 
`remarks` | dict:str | 12 | 12 | 0 183 | 397 | 2,196 | 4,764 | 217% | 21.0x | 
`repeat` | dict:int | 2,063 | 2,063 | 0 64 | 411 | 131,868 | 847,408 | 643% |  8.8x | 
`srcLn` | dict:str | 31,494 | 31,494 | 0 145 | 33 | 4,580,026 | 1,033,627 | 23% | 20.6x | 
`srcLnNum` | dict:int | 31,494 | 31,494 | 0 98 | 8 | 3,074,452 | 244,052 | 8% |  8.4x | 
`srcfile` | dict:str | 31,494 | 31,494 | 0 131 | 4 | 4,124,886 | 130,129 | 3% | 20.5x | 
`subgenre` | dict:str | 6 | 6 | 0 146 | 804 | 878 | 4,826 | 550% | 20.9x | 
`supplied` | dict:int | 246 | 246 | 0 66 | 2,875 | 16,236 | 707,148 | 4355% |  8.6x | 
`sym` | dict:str | 279,724 | 279,724 | 0 112 | 5 | 31,429,236 | 1,428,115 | 5% | 19.0x | 
`symr` | dict:str | 279,724 | 279,724 | 0 121 | 5 | 33,782,678 | 1,441,335 | 4% | 20.2x | 
`symu` | dict:str | 279,724 | 279,724 | 0 143 | 5 | 39,941,394 | 1,482,019 | 4% | 21.8x | 
`trans` | dict:int | 134 | 134 | 0 63 | 31 | 8,484 | 4,100 | 48% |  8.7x | 
`transcriber` | dict:str | 1,281 | 1,281 | 0 119 | 6 | 152,162 | 7,053 | 5% | 24.3x | 
`translation@en` | dict:str | 134 | 134 | 0 146 | 59 | 19,507 | 7,962 | 41% | 22.9x | 
`type` | dict:str | 226,668 | 226,668 | 0 130 | 4 | 29,508,162 | 956,244 | 3% | 20.0x | 
`uncertain` | dict:int | 1,194 | 1,194 | 0 59 | 525 | 70,428 | 626,604 | 890% |  8.8x | 
`volume` | dict:int | 1,284 | 1,284 | 0 59 | 6 | 75,132 | 7,140 | 10% |  8.8x | 
`TOTAL` | ALL | 3,411,847 | 3,411,847 | 0 127 | 12 | 434,511,139 | 41,154,851 | 9% | 17.2x | 


In [32]:
testFeatures('bhsa')


# bhsa 78 features

name | type | lengthOrig | lengthPack | errorselemSizeOrig | elemSizePack | sizeOrig | sizePack | sizeRatio | perfRatio | 
--- | --- | --- | --- | ------ | --- | --- | --- | --- | --- | 
`__levDown__` | tup:tup | 1,020,215 | 1,020,215 | 0 360 | 43 | 366,975,184 | 44,236,003 | 12% | 26314.1x | 
`__levUp__` | tup:tup | 1,446,799 | 1,446,799 | 0 394 | 46 | 570,684,680 | 67,252,679 | 12% | 20.2x | 
`book` | dict:str | 24,181 | 24,181 | 0 138 | 4 | 3,348,038 | 102,897 | 3% | 18.4x | 
`book@am` | dict:str | 39 | 39 | 0 153 | 73 | 5,956 | 2,842 | 48% | 28.1x | 
`book@ar` | dict:str | 39 | 39 | 0 143 | 56 | 5,580 | 2,175 | 39% | 22.3x | 
`book@bn` | dict:str | 39 | 39 | 0 148 | 68 | 5,764 | 2,638 | 46% | 11.9x | 
`book@da` | dict:str | 39 | 39 | 0 117 | 53 | 4,573 | 2,055 | 45% | 24.8x | 
`book@de` | dict:str | 39 | 39 | 0 116 | 52 | 4,527 | 2,029 | 45% | 20.9x | 
`book@el` | dict:str | 39 | 39 | 0 147 | 60 | 5,732 | 2,328 | 41% | 22.6x | 
`book@en` | dict:str | 39 | 39 | 0 115 | 52 | 4,470 | 2,029 | 45% | 22.1x | 
`book@es` | dict:str | 39 | 39 | 0 125 | 53 | 4,893 | 2,055 | 42% | 22.0x | 
`book@fa` | dict:str | 39 | 39 | 0 145 | 58 | 5,664 | 2,247 | 40% | 23.1x | 
`book@fr` | dict:str | 39 | 39 | 0 123 | 53 | 4,783 | 2,055 | 43% | 24.1x | 
`book@he` | dict:str | 39 | 39 | 0 143 | 56 | 5,586 | 2,175 | 39% | 22.3x | 
`book@hi` | dict:str | 39 | 39 | 0 146 | 65 | 5,692 | 2,522 | 44% | 23.8x | 
`book@id` | dict:str | 39 | 39 | 0 114 | 52 | 4,458 | 2,029 | 46% | 20.4x | 
`book@ja` | dict:str | 39 | 39 | 0 141 | 58 | 5,488 | 2,247 | 41% | 22.5x | 
`book@ko` | dict:str | 39 | 39 | 0 138 | 54 | 5,398 | 2,111 | 39% | 21.5x | 
`book@la` | dict:str | 39 | 39 | 0 114 | 52 | 4,455 | 2,029 | 46% | 19.7x | 
`book@nl` | dict:str | 39 | 39 | 0 116 | 52 | 4,512 | 2,029 | 45% | 20.0x | 
`book@pa` | dict:str | 39 | 39 | 0 145 | 63 | 5,670 | 2,469 | 44% | 21.9x | 
`book@pt` | dict:str | 39 | 39 | 0 125 | 53 | 4,877 | 2,055 | 42% | 19.9x | 
`book@ru` | dict:str | 39 | 39 | 0 147 | 60 | 5,736 | 2,328 | 41% | 23.1x | 
`book@sw` | dict:str | 39 | 39 | 0 115 | 53 | 4,490 | 2,055 | 46% | 20.1x | 
`book@syc` | dict:str | 39 | 39 | 0 144 | 57 | 5,632 | 2,210 | 39% | 21.6x | 
`book@tr` | dict:str | 39 | 39 | 0 124 | 53 | 4,830 | 2,082 | 43% | 23.3x | 
`book@ur` | dict:str | 39 | 39 | 0 145 | 58 | 5,644 | 2,247 | 40% | 22.0x | 
`book@yo` | dict:str | 39 | 39 | 0 122 | 53 | 4,758 | 2,082 | 44% | 19.3x | 
`book@zh` | dict:str | 39 | 39 | 0 139 | 56 | 5,423 | 2,175 | 40% | 20.8x | 
`chapter` | dict:int | 24,142 | 24,142 | 0 82 | 4 | 1,990,992 | 103,104 | 5% |  7.7x | 
`code` | dict:int | 90,688 | 90,688 | 0 98 | 4 | 8,877,400 | 386,900 | 4% |  8.6x | 
`crossref` | dict:tup | 3,783 | 3,783 | 0 417 | 63 | 1,577,780 | 239,996 | 15% | 17.2x | 
`det` | dict:str | 520,748 | 520,748 | 0 120 | 4 | 62,351,239 | 2,101,600 | 3% | 19.6x | 
`freq_lex` | dict:int | 435,817 | 435,817 | 0 96 | 4 | 41,925,360 | 1,753,888 | 4% |  8.9x | 
`function` | dict:str | 253,207 | 253,207 | 0 122 | 4 | 30,995,631 | 1,016,031 | 3% | 20.1x | 
`g_cons` | dict:str | 426,584 | 426,584 | 0 115 | 5 | 48,932,636 | 1,975,557 | 4% | 19.1x | 
`g_cons_utf8` | dict:str | 426,584 | 426,584 | 0 156 | 5 | 66,515,047 | 2,106,698 | 3% | 21.7x | 
`g_lex` | dict:str | 426,584 | 426,584 | 0 129 | 5 | 55,124,794 | 1,971,642 | 4% | 20.2x | 
`g_lex_utf8` | dict:str | 426,584 | 426,584 | 0 158 | 5 | 67,572,143 | 2,110,979 | 3% | 21.9x | 
`g_word` | dict:str | 426,584 | 426,584 | 0 132 | 7 | 56,363,862 | 3,042,603 | 5% | 19.3x | 
`g_word_utf8` | dict:str | 426,584 | 426,584 | 0 162 | 9 | 68,968,847 | 3,798,008 | 6% | 21.3x | 
`gloss` | dict:str | 435,817 | 435,817 | 0 130 | 4 | 56,603,396 | 1,823,385 | 3% | 19.9x | 
`gn` | dict:str | 426,584 | 426,584 | 0 105 | 4 | 44,653,154 | 1,752,272 | 4% | 20.2x | 
`label` | dict:str | 68,393 | 68,393 | 0 86 | 9 | 5,906,265 | 618,770 | 10% | 17.9x | 
`language` | dict:str | 435,817 | 435,817 | 0 131 | 4 | 57,151,241 | 1,752,420 | 3% | 20.0x | 
`lex` | dict:str | 435,817 | 435,817 | 0 114 | 4 | 49,850,294 | 1,834,413 | 4% | 18.7x | 
`lex_utf8` | dict:str | 426,584 | 426,584 | 0 156 | 4 | 66,700,092 | 1,834,062 | 3% | 21.7x | 
`ls` | dict:str | 426,961 | 426,961 | 0 130 | 4 | 55,555,457 | 1,752,553 | 3% | 20.3x | 
`mother` | dict:tup | 182,174 | 182,174 | 0 170 | 25 | 30,889,352 | 4,588,592 | 15% | 16.5x | 
`nametype` | dict:str | 38,184 | 38,184 | 0 117 | 46 | 4,451,056 | 1,752,685 | 39% | 21.5x | 
`nu` | dict:str | 426,584 | 426,584 | 0 128 | 4 | 54,714,362 | 1,752,272 | 3% | 20.4x | 
`number` | dict:int | 1,254,393 | 1,254,393 | 0 80 | 4 | 99,760,484 | 5,337,592 | 5% |  8.7x | 
`oslots` | tup:tup | 1,020,215 | 1,020,215 | 0 227 | 19 | 231,817,308 | 19,272,595 | 8% | 10205.6x | 
`otype` | tup:str | 1,020,215 | 1,020,215 | 0 8 | 4 | 8,162,449 | 4,093,245 | 50% | 20.6x | 
`pdp` | dict:str | 426,584 | 426,584 | 0 130 | 4 | 55,495,839 | 1,752,353 | 3% | 20.8x | 
`phono` | dict:str | 426,584 | 426,584 | 0 156 | 7 | 66,334,223 | 2,856,249 | 4% | 21.9x | 
`phono_trailer` | dict:str | 426,584 | 426,584 | 0 80 | 4 | 33,963,914 | 1,752,264 | 5% | 18.6x | 
`prs_gn` | dict:str | 426,584 | 426,584 | 0 127 | 4 | 54,082,296 | 1,752,272 | 3% | 19.1x | 
`prs_nu` | dict:str | 426,584 | 426,584 | 0 131 | 4 | 55,851,462 | 1,752,272 | 3% | 21.8x | 
`prs_ps` | dict:str | 426,584 | 426,584 | 0 131 | 4 | 55,851,462 | 1,752,272 | 3% | 21.4x | 
`ps` | dict:str | 426,584 | 426,584 | 0 128 | 4 | 54,757,762 | 1,752,272 | 3% | 20.1x | 
`qere` | dict:str | 1,892 | 1,892 | 0 123 | 881 | 232,529 | 1,667,529 | 717% | 19.9x | 
`qere_trailer` | dict:str | 1,892 | 1,892 | 0 71 | 872 | 133,515 | 1,650,076 | 1236% | 20.1x | 
`qere_trailer_utf8` | dict:str | 1,892 | 1,892 | 0 75 | 872 | 141,415 | 1,650,076 | 1167% | 19.9x | 
`qere_utf8` | dict:str | 1,892 | 1,892 | 0 153 | 887 | 290,147 | 1,677,734 | 578% | 23.0x | 
`rank_lex` | dict:int | 435,817 | 435,817 | 0 83 | 4 | 36,117,624 | 1,754,012 | 5% |  8.7x | 
`rela` | dict:str | 722,681 | 722,681 | 0 137 | 4 | 99,163,187 | 3,023,653 | 3% | 20.3x | 
`sp` | dict:str | 435,817 | 435,817 | 0 129 | 4 | 56,242,406 | 1,752,517 | 3% | 20.7x | 
`st` | dict:str | 426,584 | 426,584 | 0 106 | 4 | 45,429,172 | 1,752,264 | 4% | 18.7x | 
`trailer` | dict:str | 426,584 | 426,584 | 0 80 | 4 | 34,246,069 | 1,752,332 | 5% | 18.5x | 
`trailer_utf8` | dict:str | 426,584 | 426,584 | 0 89 | 4 | 38,127,141 | 1,752,342 | 5% | 18.4x | 
`txt` | dict:str | 88,121 | 88,121 | 0 115 | 4 | 10,145,213 | 364,212 | 4% | 20.9x | 
`typ` | dict:str | 699,557 | 699,557 | 0 140 | 4 | 97,701,785 | 2,846,001 | 3% | 19.0x | 
`verse` | dict:int | 23,213 | 23,213 | 0 85 | 4 | 1,965,708 | 97,048 | 5% |  8.7x | 
`voc_lex` | dict:str | 435,817 | 435,817 | 0 129 | 4 | 56,295,121 | 1,841,554 | 3% | 21.2x | 
`voc_lex_utf8` | dict:str | 435,817 | 435,817 | 0 158 | 4 | 69,038,792 | 1,894,397 | 3% | 21.6x | 
`vs` | dict:str | 426,584 | 426,584 | 0 128 | 4 | 54,753,857 | 1,752,462 | 3% | 20.2x | 
`vt` | dict:str | 426,584 | 426,584 | 0 129 | 4 | 54,819,172 | 1,752,332 | 3% | 20.3x | 
`TOTAL` | ALL | 21,381,252 | 21,381,252 | 0 152 | 10 | 3,249,732,945 | 222,771,234 | 7% | 487.4x | 


In [33]:
testFeatures('quran')


# quran 41 features

name | type | lengthOrig | lengthPack | errorselemSizeOrig | elemSizePack | sizeOrig | sizePack | sizeRatio | perfRatio | 
--- | --- | --- | --- | ------ | --- | --- | --- | --- | --- | 
`__levDown__` | tup:tup | 90,063 | 90,063 | 0 295 | 36 | 26,535,564 | 3,261,527 | 12% | 17.2x | 
`__levUp__` | tup:tup | 218,282 | 218,282 | 0 367 | 34 | 80,171,692 | 7,450,295 | 9% | 18.3x | 
`a` | dict:str | 513 | 513 | 0 121 | 959 | 62,133 | 492,052 | 792% | 20.5x | 
`ascii` | dict:str | 128,219 | 128,219 | 0 122 | 5 | 15,662,584 | 669,443 | 4% | 20.2x | 
`ax` | dict:str | 513 | 513 | 0 126 | 959 | 64,692 | 492,070 | 761% | 20.6x | 
`case` | dict:str | 31,914 | 31,914 | 0 127 | 16 | 4,062,076 | 521,910 | 13% | 20.3x | 
`component` | dict:str | 128,219 | 128,219 | 0 123 | 4 | 15,729,323 | 521,897 | 3% | 19.1x | 
`definite` | dict:int | 8,672 | 8,672 | 0 62 | 60 | 537,852 | 521,876 | 97% |  8.4x | 
`f` | dict:str | 3,001 | 3,001 | 0 135 | 174 | 404,196 | 521,885 | 129% | 20.6x | 
`formation` | dict:str | 8,977 | 8,977 | 0 107 | 58 | 960,011 | 521,942 | 54% | 19.7x | 
`fx` | dict:str | 3,001 | 3,001 | 0 136 | 174 | 408,151 | 521,930 | 128% | 21.6x | 
`gn` | dict:str | 66,606 | 66,606 | 0 67 | 8 | 4,486,612 | 521,864 | 12% | 19.6x | 
`interjection` | dict:str | 5 | 5 | 0 133 | 394 | 665 | 1,968 | 296% | 20.9x | 
`l` | dict:str | 3,849 | 3,849 | 0 125 | 136 | 479,282 | 521,885 | 109% | 20.7x | 
`lemma` | dict:str | 79,440 | 79,440 | 0 116 | 8 | 9,187,247 | 611,379 | 7% | 20.3x | 
`lx` | dict:str | 1,398 | 1,398 | 0 138 | 373 | 192,491 | 521,910 | 271% | 20.4x | 
`mood` | dict:str | 2,748 | 2,748 | 0 140 | 190 | 383,712 | 522,049 | 136% | 20.9x | 
`n` | dict:str | 243 | 243 | 0 123 | 1,795 | 29,975 | 436,068 | 1455% | 20.4x | 
`name` | dict:str | 114 | 114 | 0 155 | 32 | 17,646 | 3,661 | 21% | 23.5x | 
`name@en` | dict:str | 114 | 114 | 0 130 | 32 | 14,823 | 3,661 | 25% | 20.6x | 
`nameAscii` | dict:str | 114 | 114 | 0 124 | 26 | 14,172 | 2,995 | 21% | 20.7x | 
`nameTrans` | dict:str | 114 | 114 | 0 127 | 29 | 14,466 | 3,346 | 23% | 19.3x | 
`nu` | dict:str | 57,316 | 57,316 | 0 74 | 9 | 4,226,542 | 521,880 | 12% | 18.1x | 
`number` | dict:int | 213,450 | 213,450 | 0 77 | 4 | 16,481,448 | 902,584 | 5% |  8.4x | 
`order` | dict:int | 114 | 114 | 0 97 | 20 | 11,088 | 2,332 | 21% |  8.8x | 
`oslots` | tup:tup | 90,063 | 90,063 | 0 531 | 63 | 47,865,640 | 5,632,507 | 12% | 16.0x | 
`otype` | tup:str | 90,063 | 90,063 | 0 8 | 4 | 721,083 | 362,923 | 50% | 20.3x | 
`pos` | dict:str | 128,219 | 128,219 | 0 125 | 4 | 15,999,835 | 521,993 | 3% | 19.6x | 
`posx` | dict:str | 60,353 | 60,353 | 0 129 | 9 | 7,809,599 | 522,422 | 7% | 22.3x | 
`ps` | dict:str | 44,092 | 44,092 | 0 87 | 12 | 3,856,270 | 521,880 | 14% | 19.4x | 
`root` | dict:str | 49,968 | 49,968 | 0 132 | 11 | 6,619,080 | 533,950 | 8% | 20.6x | 
`sp` | dict:str | 3,870 | 3,870 | 0 119 | 135 | 461,030 | 521,892 | 113% | 20.8x | 
`space` | dict:str | 128,219 | 128,219 | 0 69 | 4 | 8,833,207 | 521,860 | 6% | 18.6x | 
`tense` | dict:str | 23,555 | 23,555 | 0 141 | 22 | 3,324,321 | 521,930 | 16% | 20.4x | 
`translation@en` | dict:str | 6,236 | 6,236 | 0 249 | 137 | 1,553,078 | 854,923 | 55% | 20.7x | 
`translation@nl` | dict:str | 6,236 | 6,236 | 0 264 | 154 | 1,645,936 | 958,407 | 58% | 19.6x | 
`type` | dict:str | 129 | 129 | 0 120 | 15 | 15,510 | 1,988 | 13% | 20.7x | 
`unicode` | dict:str | 128,219 | 128,219 | 0 152 | 6 | 19,553,247 | 772,344 | 4% | 21.7x | 
`voice` | dict:str | 4,665 | 4,665 | 0 115 | 112 | 536,446 | 521,876 | 97% | 20.8x | 
`w` | dict:str | 9,594 | 9,594 | 0 118 | 54 | 1,135,113 | 521,910 | 46% | 20.0x | 
`wx` | dict:str | 9,566 | 9,566 | 0 120 | 55 | 1,144,894 | 521,941 | 46% | 20.7x | 
`TOTAL` | ALL | 1,830,046 | 1,830,046 | 0 165 | 19 | 301,212,732 | 33,889,155 | 11% | 19.3x | 


# Testing mild memory reduction

Before the optimization.

In [125]:
footPrint('banks')


# banks 17 features

feature | members | size in bytes
--- | --- | ---
__boundary__ | 2 | 3,736
__levDown__ | 18 | 1,364
__levUp__ | 117 | 10,576
__levels__ | 5 | 1,108
__order__ | 117 | 572
__rank__ | 117 | 572
__sections__ | 2 | 1,728
__structure__ | 6 | 9,468
author | 1 | 330
gap | 7 | 592
letters | 99 | 12,826
number | 17 | 1,348
oslots | 19 | 6,948
otype | 20 | 508
punc | 17 | 1,324
terminator | 12 | 1,184
title | 1 | 333
TOTAL | 577 | 54,517

After the optimization

In [128]:
footPrint('banks')

   |      |     0.00s C __levels__           from otype, oslots, otext
   |      |     0.00s C __order__            from otype, oslots, __levels__
   |      |     0.00s C __rank__             from otype, __order__
   |      |     0.00s C __levUp__            from otype, oslots, __levels__, __rank__
   |      |     0.00s C __levDown__          from otype, __levUp__, __rank__
   |      |     0.00s C __boundary__         from otype, oslots, __rank__
   |      |     0.00s C __sections__         from otype, oslots, otext, __levUp__, __levels__, title, number, number
   |      |     0.00s C __structure__        from otype, oslots, otext, __rank__, __levUp__, title, number, number, number



# banks 17 features

feature | members | size in bytes
--- | --- | ---
__boundary__ | 2 | 3,736
__levDown__ | 18 | 1,364
__levUp__ | 117 | 2,960
__levels__ | 5 | 1,108
__order__ | 117 | 532
__rank__ | 117 | 588
__sections__ | 2 | 1,728
__structure__ | 6 | 9,468
author | 1 | 330
gap | 7 | 592
letters | 99 | 11,581
number | 17 | 1,348
oslots | 19 | 6,948
otype | 20 | 508
punc | 17 | 1,324
terminator | 12 | 1,184
title | 1 | 333
TOTAL | 577 | 45,632

Before the optimization.

In [129]:
footPrint('bhsa')


# bhsa 83 features

feature | members | size in bytes
--- | --- | ---
__boundary__ | 2 | 112,432,776
__levDown__ | 1,020,215 | 366,975,184
__levUp__ | 1,446,799 | 570,684,680
__levels__ | 13 | 2,942
__order__ | 1,446,799 | 6,148,968
__rank__ | 1,446,799 | 6,148,968
__sections__ | 2 | 1,721,128
book | 24,181 | 3,348,038
book@am | 39 | 5,956
book@ar | 39 | 5,580
book@bn | 39 | 5,764
book@da | 39 | 4,573
book@de | 39 | 4,527
book@el | 39 | 5,732
book@en | 39 | 4,470
book@es | 39 | 4,893
book@fa | 39 | 5,664
book@fr | 39 | 4,783
book@he | 39 | 5,586
book@hi | 39 | 5,692
book@id | 39 | 4,458
book@ja | 39 | 5,488
book@ko | 39 | 5,398
book@la | 39 | 4,455
book@nl | 39 | 4,512
book@pa | 39 | 5,670
book@pt | 39 | 4,877
book@ru | 39 | 5,736
book@sw | 39 | 4,490
book@syc | 39 | 5,632
book@tr | 39 | 4,830
book@ur | 39 | 5,644
book@yo | 39 | 4,758
book@zh | 39 | 5,423
chapter | 24,142 | 1,990,992
code | 90,688 | 8,877,400
crossref | 3,783 | 2,872,548
det | 520,748 | 62,351,239
freq_lex | 435,817 | 41,925,360
function | 253,207 | 30,995,631
g_cons | 426,584 | 48,932,636
g_cons_utf8 | 426,584 | 66,515,047
g_lex | 426,584 | 55,124,794
g_lex_utf8 | 426,584 | 67,572,143
g_word | 426,584 | 56,363,862
g_word_utf8 | 426,584 | 68,968,847
gloss | 435,817 | 56,603,396
gn | 426,584 | 44,653,154
label | 68,393 | 5,906,265
language | 435,817 | 57,151,241
lex | 435,817 | 49,850,294
lex_utf8 | 426,584 | 66,700,092
ls | 426,961 | 55,555,457
mother | 182,174 | 61,494,584
nametype | 38,184 | 4,451,056
nu | 426,584 | 54,714,362
number | 1,254,393 | 99,760,484
oslots | 1,020,216 | 231,817,344
otype | 1,020,217 | 8,162,546
pdp | 426,584 | 55,495,839
phono | 426,584 | 66,334,223
phono_trailer | 426,584 | 33,963,914
prs_gn | 426,584 | 54,082,296
prs_nu | 426,584 | 55,851,462
prs_ps | 426,584 | 55,851,462
ps | 426,584 | 54,757,762
qere | 1,892 | 232,529
qere_trailer | 1,892 | 133,515
qere_trailer_utf8 | 1,892 | 141,415
qere_utf8 | 1,892 | 290,147
rank_lex | 435,817 | 36,117,624
rela | 722,681 | 99,163,187
sp | 435,817 | 56,242,406
st | 426,584 | 45,429,172
trailer | 426,584 | 34,246,069
trailer_utf8 | 426,584 | 38,127,141
txt | 88,121 | 10,145,213
typ | 699,557 | 97,701,785
verse | 23,213 | 1,965,708
voc_lex | 435,817 | 56,295,121
voc_lex_utf8 | 435,817 | 69,038,792
vs | 426,584 | 54,753,857
vt | 426,584 | 54,819,172
TOTAL | 24,274,870 | 3,408,087,860

After the optimization

In [130]:
footPrint('bhsa')

   |      |     0.68s C __levels__           from otype, oslots, otext
   |      |       12s C __order__            from otype, oslots, __levels__
   |      |     0.61s C __rank__             from otype, __order__
   |      |       12s C __levUp__            from otype, oslots, __levels__, __rank__
   |      |     7.94s C __levDown__          from otype, __levUp__, __rank__
   |      |     2.57s C __boundary__         from otype, oslots, __rank__
   |      |     0.12s C __sections__         from otype, oslots, otext, __levUp__, __levels__, book, chapter, verse



# bhsa 83 features

feature | members | size in bytes
--- | --- | ---
__boundary__ | 2 | 112,432,776
__levDown__ | 1,020,215 | 150,285,088
__levUp__ | 1,446,799 | 321,340,208
__levels__ | 13 | 2,942
__order__ | 1,446,799 | 5,787,260
__rank__ | 1,446,799 | 5,887,292
__sections__ | 2 | 1,720,036
book | 24,181 | 1,990,063
book@am | 39 | 7,086
book@ar | 39 | 6,024
book@bn | 39 | 6,684
book@da | 39 | 4,624
book@de | 39 | 4,561
book@el | 39 | 6,326
book@en | 39 | 4,470
book@es | 39 | 5,059
book@fa | 39 | 6,195
book@fr | 39 | 4,900
book@he | 39 | 6,040
book@hi | 39 | 6,500
book@id | 39 | 4,458
book@ja | 39 | 6,016
book@ko | 39 | 5,791
book@la | 39 | 4,455
book@nl | 39 | 4,526
book@pa | 39 | 6,433
book@pt | 39 | 5,048
book@ru | 39 | 6,331
book@sw | 39 | 4,490
book@syc | 39 | 6,132
book@tr | 39 | 4,962
book@ur | 39 | 6,154
book@yo | 39 | 4,860
book@zh | 39 | 5,786
chapter | 24,142 | 1,990,992
code | 90,688 | 7,790,440
crossref | 3,783 | 2,872,548
det | 520,748 | 35,552,715
freq_lex | 435,817 | 33,179,140
function | 253,207 | 17,577,197
g_cons | 426,584 | 34,229,809
g_cons_utf8 | 426,584 | 35,235,418
g_lex | 426,584 | 34,079,628
g_lex_utf8 | 426,584 | 35,018,177
g_word | 426,584 | 38,366,709
g_word_utf8 | 426,584 | 43,107,744
gloss | 435,817 | 33,519,892
gn | 426,584 | 32,916,175
label | 68,393 | 5,906,265
language | 435,817 | 33,174,603
lex | 435,817 | 33,649,109
lex_utf8 | 426,584 | 33,512,214
ls | 426,961 | 32,927,319
mother | 182,174 | 51,887,336
nametype | 38,184 | 2,380,469
nu | 426,584 | 32,916,228
number | 1,254,393 | 77,891,588
oslots | 1,020,216 | 231,817,344
otype | 1,020,217 | 8,162,546
pdp | 426,584 | 32,916,709
phono | 426,584 | 66,334,223
phono_trailer | 426,584 | 33,963,914
prs_gn | 426,584 | 32,916,175
prs_nu | 426,584 | 32,916,177
prs_ps | 426,584 | 32,916,228
ps | 426,584 | 32,916,228
qere | 1,892 | 200,647
qere_trailer | 1,892 | 127,053
qere_trailer_utf8 | 1,892 | 127,142
qere_utf8 | 1,892 | 263,799
rank_lex | 435,817 | 33,179,248
rela | 722,681 | 62,179,423
sp | 435,817 | 33,175,233
st | 426,584 | 32,916,169
trailer | 426,584 | 32,916,599
trailer_utf8 | 426,584 | 32,916,939
txt | 88,121 | 7,717,641
typ | 699,557 | 61,533,700
verse | 23,213 | 1,965,708
voc_lex | 435,817 | 33,624,383
voc_lex_utf8 | 435,817 | 33,998,144
vs | 426,584 | 32,917,336
vt | 426,584 | 32,916,443
TOTAL | 24,274,870 | 2,226,806,442

# Now with arrays instead of tuples

In [9]:
footPrint('bhsa')


# bhsa 83 features

feature | members | size in bytes
--- | --- | ---
__boundary__ | 2 | 77,627,912
__levDown__ | 1,020,215 | 120,578,296
__levUp__ | 1,446,799 | 171,509,072
__levels__ | 13 | 2,942
__order__ | 1,446,799 | 6,148,968
__rank__ | 1,446,799 | 6,148,968
__sections__ | 2 | 1,721,128
book | 24,181 | 1,990,063
book@am | 39 | 5,956
book@ar | 39 | 5,580
book@bn | 39 | 5,764
book@da | 39 | 4,573
book@de | 39 | 4,527
book@el | 39 | 5,732
book@en | 39 | 4,470
book@es | 39 | 4,893
book@fa | 39 | 5,664
book@fr | 39 | 4,783
book@he | 39 | 5,586
book@hi | 39 | 5,692
book@id | 39 | 4,458
book@ja | 39 | 5,488
book@ko | 39 | 5,398
book@la | 39 | 4,455
book@nl | 39 | 4,512
book@pa | 39 | 5,670
book@pt | 39 | 4,877
book@ru | 39 | 5,736
book@sw | 39 | 4,490
book@syc | 39 | 5,632
book@tr | 39 | 4,830
book@ur | 39 | 5,644
book@yo | 39 | 4,758
book@zh | 39 | 5,341
chapter | 24,142 | 1,990,992
code | 90,688 | 8,877,400
crossref | 3,783 | 2,872,548
det | 520,748 | 35,552,715
freq_lex | 435,817 | 41,925,360
function | 253,207 | 17,577,197
g_cons | 426,584 | 34,229,809
g_cons_utf8 | 426,584 | 34,967,915
g_lex | 426,584 | 34,079,628
g_lex_utf8 | 426,584 | 34,738,575
g_word | 426,584 | 38,366,709
g_word_utf8 | 426,584 | 41,387,193
gloss | 435,817 | 33,519,892
gn | 426,584 | 32,916,175
label | 68,393 | 5,906,265
language | 435,817 | 33,174,603
lex | 435,817 | 33,649,109
lex_utf8 | 426,584 | 33,450,178
ls | 426,961 | 32,927,319
mother | 182,174 | 51,887,336
nametype | 38,184 | 2,380,469
nu | 426,584 | 32,916,228
number | 1,254,393 | 99,760,484
oslots | 3 | 105,561,100
otype | 4 | 8,162,638
pdp | 426,584 | 32,916,709
phono | 426,584 | 38,354,048
phono_trailer | 426,584 | 32,916,170
prs_gn | 426,584 | 32,916,175
prs_nu | 426,584 | 32,916,177
prs_ps | 426,584 | 32,916,228
ps | 426,584 | 32,916,228
qere | 1,892 | 200,647
qere_trailer | 1,892 | 127,053
qere_trailer_utf8 | 1,892 | 127,131
qere_utf8 | 1,892 | 241,193
rank_lex | 435,817 | 36,117,624
rela | 722,681 | 62,179,423
sp | 435,817 | 33,175,233
st | 426,584 | 32,916,169
trailer | 426,584 | 32,916,599
trailer_utf8 | 426,584 | 32,916,877
txt | 88,121 | 7,717,641
typ | 699,557 | 61,533,700
verse | 23,213 | 1,965,708
voc_lex | 435,817 | 33,624,383
voc_lex_utf8 | 435,817 | 33,881,864
vs | 426,584 | 32,917,336
vt | 426,584 | 32,916,443
TOTAL | 22,234,444 | 1,889,966,454

In [19]:
patres = '~/github/pthu/pilot/combined/tf/1.0'
footPrint(patres, name='patres', fabric=True)


# patres 463 features

feature | members | size in bytes
--- | --- | ---
__boundary__ | 2 | 723,529,420
__levDown__ | 481,797 | 56,739,020
__levUp__ | 5,425,803 | 121,748,552
__levels__ | 448 | 112,221
__order__ | 5,425,803 | 23,059,736
__rank__ | 5,425,803 | 23,059,736
__sections__ | 2 | 19,052
__structure__ | 6 | 165,563
_book | 163 | 21,112
_sentence | 392,112 | 41,265,568
ab | 2 | 324
add | 2,142 | 150,208
altsection | 30 | 2,220
argument | 10 | 928
beta | 4,944,006 | 323,947,646
beta_plain | 4,944,006 | 320,913,828
bibl | 140 | 16,913
book | 216 | 16,912
chapter | 7,850 | 535,493
comment | 143 | 21,240
commentary | 12 | 1,646
date | 15 | 2,168
del | 689 | 66,032
div | 551 | 35,233
div-#Agathius | 2 | 352
div-#Apostolos | 4 | 464
div-#Basil | 11 | 1,124
div-#Caesarius | 1 | 296
div-#Chrysologus | 3 | 352
div-#Chrysoremonos | 1 | 296
div-#Chrysostom | 190 | 15,004
div-#Chysoreithrologus | 1 | 296
div-#Clement | 2 | 324
div-#Cyril | 65 | 4,464
div-#D | 4 | 464
div-#Didymus | 1 | 296
div-#Diodorus | 3 | 380
div-#Euthalius | 2 | 352
div-#Gennadius | 61 | 4,324
div-#Isidore | 11 | 1,152
div-#Maximus | 7 | 676
div-#Methodius | 2 | 352
div-#Monachus | 36 | 2,360
div-#Nyssa | 4 | 464
div-#Oecumenius | 13 | 1,208
div-#Origen | 8 | 732
div-#Patara | 1 | 296
div-#Photius | 22 | 2,052
div-#Same | 3 | 408
div-#Severianus | 16 | 1,292
div-#Theodore | 10 | 816
div-#Theodoret | 131 | 8,764
div-pr | 2 | 347
div-urn:cts:greekLit:tlg0031.tlg001:1.18 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:11.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:11.12 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:12.14 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:12.15 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:13.44 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:13.47 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:18.20 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:18.21 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:18.5 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:21.33 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:24.45 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:25.1 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:27.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:27.45 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:28.18 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:7.5 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:7.6 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:7.9 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg001:8.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.1 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.10 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.12 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.13 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.14 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.22 | 2 | 324
div-urn:cts:greekLit:tlg0031.tlg003:1.23 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.24 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.25 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.26 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.27 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.3 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.36 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.4 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.5 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.6 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.63 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.7 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.8 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:1.9 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.10 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.38 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.39 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.40 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.41 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:10.42 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.2 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.3 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.34 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.35 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.36 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.5-11.7 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:11.51 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:12.6 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:12.7 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:13.20 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:13.21 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:13.29 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:14.12 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:14.13 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:14.14 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:14.16 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:14.19 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:14.20 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:15.23 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:16.16 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:17.12 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:17.13 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:18.20 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.29 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.30 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.31 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.32 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.33 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.34 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.35 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:19.36 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:2.49 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.19 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.20 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.21 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.22 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.23 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.24 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.25 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:20.26 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:3.1 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:3.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:3.3 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:3.5 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:4.40 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:4.9 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:7.22 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.16 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.17 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.38 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.39 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.48 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.52 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:8.53 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.26 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.28 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.29 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.30 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.31 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.35 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.36 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.42 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.43 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.44 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.56 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg003:9.58 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.1-1.3 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.10 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.11 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.12 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.13 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.14-1.15 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.16 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.17-1.19 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.20-1.23 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.24-1.25 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.4 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.5-1.6 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.7 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.8 | 1 | 296
div-urn:cts:greekLit:tlg0031.tlg026:1.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg001:19.25-19.26 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg001:23.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg001:37.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg001:43.22 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg001:46.34 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg001:7.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg002:15.25 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg002:20.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg002:28.30 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg002:31.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:12.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:13.45 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:23.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:23.40 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:27.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:27.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:4.27 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg003:4.28 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:11.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:12.1-12.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:14.36 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:22.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:22.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:23.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:24.17 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg004:24.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:1.31 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:14.19 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:20.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:20.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:20.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:20.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:22.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:23.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:23.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:23.14 | 2 | 400
div-urn:cts:greekLit:tlg0527.tlg005:23.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:24.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:27.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:27.17 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:27.18 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:27.19 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:27.26 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:3.18 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:3.27-3.28 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:33.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg005:4.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg006:3.14 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg006:6.15-6.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg006:8.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg006:8.26 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg008:15.19 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg008:20.21 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:13.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:13.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:13.5-13.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:15.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:150.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:16.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:17.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:17.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:17.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:22.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:22.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:23.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:23.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:23.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:23.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:23.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:27.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:27.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:27.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:27.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.14 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.17 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.21 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.22 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.23 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.24 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.25 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:36.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:41.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:41.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:50.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:50.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:76.30 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:76.31 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:76.44 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:80.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:9.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:9.33 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:9.38 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:9.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg027:9.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:1.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:1.18 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:1.19 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:19.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:19.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:19.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:19.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:19.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:19.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:2.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:2.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:2.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:2.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:2.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:2.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.17 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.18 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.20 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.22 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.24 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.25 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.27 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.29 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:20.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:21.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.14 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.18 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.19 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.20 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.22 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.23 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.24 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.25 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.26 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.28 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:22.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:23.2 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:24.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:27.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:27.2 | 2 | 398
div-urn:cts:greekLit:tlg0527.tlg032:27.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:27.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:28.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:28.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:28.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:28.18 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:28.21 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.12 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.16 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:29.9 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:3.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.1 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.19 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.20 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.22 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:32.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:35.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:35.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:35.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:35.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.11 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.25 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.26 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.3 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.7 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:40.8 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.13 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.14 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.15 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.4 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.5 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:41.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg032:42.10 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg053:30.25 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg053:32.23 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg053:32.6 | 1 | 296
div-urn:cts:greekLit:tlg0527.tlg053:44.1 | 1 | 296
div2 | 362 | 31,163
div3 | 1,865 | 128,060
epistle | 1 | 296
foreign | 83 | 4,711
fragment | 3,432 | 251,665
gap | 126 | 8,591
haeresis | 132 | 11,924
head | 2,293 | 327,652
hi | 5 | 438
homilia | 59 | 5,966
l | 8,188 | 672,812
lb | 76,965 | 4,797,942
lemma | 4,944,241 | 317,214,660
letter | 1 | 292
lg | 1,216 | 89,328
line | 1,706 | 122,708
main | 4,944,006 | 334,281,918
milestone | 3,972 | 431,765
milestone-altedition | 381 | 39,836
milestone-altnumbering | 668 | 54,099
milestone-altpage1 | 115 | 14,728
milestone-altpage2 | 120 | 15,288
milestone-altref | 506 | 45,337
milestone-chapter | 332 | 20,380
milestone-fabrnumbering | 527 | 47,424
milestone-fabrsection | 1,387 | 112,810
milestone-hnumbering | 394 | 56,467
milestone-manuscriptpage | 462 | 55,378
milestone-mignepage | 769 | 98,331
milestone-ms | 34 | 3,904
milestone-mspage | 271 | 30,153
milestone-pageseg | 2,352 | 140,030
milestone-rnumbering | 444 | 54,072
milestone-stephnumbering | 492 | 46,080
milestone-vignumbering | 837 | 83,840
milestone-volume | 2 | 352
note | 29,352 | 10,188,325
num | 564 | 42,132
orig | 4,944,006 | 353,639,016
oslots | 3 | 286,172,440
otype | 4 | 3,891,493
p | 70,948 | 4,957,736
page | 504 | 42,688
paragraph | 4,230 | 275,136
part | 2 | 352
pb | 21,288 | 1,507,884
pb-M | 5 | 645
pb-P | 5 | 645
pb-alt | 485 | 45,688
pb-r | 200 | 20,520
pb-v | 201 | 20,548
plain | 4,944,006 | 330,047,768
post | 4,943,751 | 306,403,860
pre | 62,032 | 4,377,119
q | 1,762 | 191,947
quote | 189 | 26,506
section | 27,351 | 2,100,558
seg | 98 | 9,240
sentence | 23 | 2,472
sic | 2 | 352
sp | 219 | 21,584
speaker | 218 | 21,528
subsection | 3,239 | 238,812
text | 1 | 296
verse | 143 | 9,716
TOTAL | 52,106,293 | 3,599,140,238